# TMF Guru demo

## Dependencies

In [20]:
!pip install ratelimit ratelimiter python-dotenv

In [21]:
from ratelimiter import RateLimiter
import pandas as pd

## Environment Variables

In [22]:
 import os
 from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

PROJECT_ID = os.getenv('PROJECT_ID')
LOCATION = os.getenv('AGENT_LOCATION') # "global" # "us-central1"
REASONING_ENGINE_ID = "6186010747542175744"


## TMF AVIA - Response parser boiler plate


In [23]:
# doc_type values - pdf / web
def parse_search_tool_response(response, doc_type="pdf"):
  tool_response = {}
  # check if tool has a answer in the response
  if "answer" in response:
    answer_object = response["answer"]
    # print(answer_object.keys())
    # successful retrieval
    if "state" in answer_object and answer_object["state"] == "SUCCEEDED":
      answerText = answer_object["answerText"] if "answerText" in answer_object else ""
      # citations = answer_object["citations"] if "citations" in answer_object else []
      # parse references
      references = []
      references_raw_list = answer_object["references"] if "references" in answer_object else []
      for ref in references_raw_list:
        chunk_info = ref["chunkInfo"] if "chunkInfo" in ref else  ""
        if "" != chunk_info and "documentMetadata" in chunk_info:
          document_metadata = chunk_info["documentMetadata"]
          reference = {}
          title = document_metadata["title"] if "title" in document_metadata else ""
          # TODO change uri from gs:// to http://
          uri = document_metadata["uri"] if "uri" in document_metadata else ""
          reference["title"] = title
          reference["uri"] = uri
          docId = ""
          if doc_type == "pdf":
            if "structData" in document_metadata:
              struct_data = document_metadata["structData"]
              docId = document_metadata["structData"]["DocId"] if "DocId" in document_metadata["structData"] else ""
              reference["docId"] = docId
          references.append(reference)

      related_questions = answer_object["relatedQuestions"] if "relatedQuestions" in answer_object else []
      tool_response = {
        "answerText": answerText,
        "references": references,
        "related_questions": related_questions
      }
    return tool_response


def parse_agent_response(response):
  formated_agent_respone = {}
  if "output" in response:
    agent_output_response = response["output"]
    is_codegen = False
    formated_agent_respone["input"] = agent_output_response["input"]
    formated_agent_respone["output"] = agent_output_response["output"]
    # hacky solution
    if ".zip" in agent_output_response["output"]:
      is_codegen = True
    if "intermediate_steps" in agent_output_response:
      intermediate_steps = agent_output_response["intermediate_steps"]
      if len(intermediate_steps) > 0:
        intermediate_steps = intermediate_steps[0]
        if len(intermediate_steps) == 2:
          tool_responses = intermediate_steps[1]
          if len(tool_responses) == 3:
            pdf_response = tool_responses[0]
            web_response = tool_responses[1]
            image_response = tool_responses[2]
            # parse pdf and web search tool response
            pdf_response_json = parse_search_tool_response(pdf_response, doc_type="pdf")
            web_response_json = parse_search_tool_response(web_response, doc_type="web")
            # parse image search tool response
            image_response_json = {}
            if "summary" in image_response:
              if image_response["summary"] != "":
                image_summary = image_response["summary"]
                image_response_json["summary"] = image_summary

            image_docs = []
            if "results" in image_response:
              if len(image_response["results"]) > 0:
                for image_doc in image_response["results"]:
                  if "document" in image_doc:
                    if "derivedStructData" in image_doc["document"]:
                      title = image_doc["document"]["derivedStructData"]["title"]
                      link = image_doc["document"]["derivedStructData"]["link"]
                      image_metadata = image_doc["document"]["derivedStructData"]["image"]
                      context_link = image_metadata["contextLink"]
                      image = {}
                      image["title"] = title
                      image["link"] = link
                      image["contextLink"] = context_link
                      image_docs.append(image)
                image_response_json["images"] = image_docs

            formated_agent_respone["pdf"] = pdf_response_json
            formated_agent_respone["web"] = web_response_json
            formated_agent_respone["image"] = image_response_json
          elif len(tool_responses) == 1:
            code_gen_response = tool_responses
            if "result" in tool_responses:
              code_gen_response = tool_responses["result"]
              formated_agent_respone["code"] = code_gen_response
          else:
            print("no of tool response: ", len(tool_responses))
  return formated_agent_respone

def build_agent_response_dict(formatted_agent_response):
  agent_response_dict = {}
  if "pdf" in formatted_agent_response:
    agent_response_dict["pdf search summary"] = formatted_agent_response["pdf"]["answerText"]
    # agent_response_dict["pdf citations"] = formatted_agent_response["pdf"]["citations"]
    agent_response_dict["pdf references"] = formatted_agent_response["pdf"]["references"]
  else:
    agent_response_dict["pdf search summary"] = ""
    # agent_response_dict["pdf citations"] = []
    agent_response_dict["pdf references"] = []

  if "web" in formatted_agent_response:
    agent_response_dict["web search summary"] = formatted_agent_response["web"]["answerText"]
    # agent_response_dict["web citations"] = formatted_agent_response["web"]["citations"]
    agent_response_dict["web references"] = formatted_agent_response["web"]["references"]
  else:
    agent_response_dict["web search summary"] = ""
    # agent_response_dict["web citations"] = []
    agent_response_dict["web references"] = []

  if "image" in formatted_agent_response:
    agent_response_dict["images"] = formatted_agent_response["image"]["images"] if "image" in formatted_agent_response and "images" in formatted_agent_response["image"] else []
  else:
    agent_response_dict["images"] = []

  if "code" in formatted_agent_response:
    agent_response_dict["code"] = formatted_agent_response["code"] if "code" in formatted_agent_response else ""
  else:
    agent_response_dict["code"] = ""

  agent_response_dict["agent summary"] = formatted_agent_response["output"] if "output" in formatted_agent_response else ""
  agent_response_dict["input"] = formatted_agent_response["input"] if "input" in formatted_agent_response else ""
  return agent_response_dict


## TMF AVIA - Curl client python example

In [24]:
import google.auth
import json
import requests

def call_tmf_aiva_agent(query_str, token):
  """Makes a POST request to the specified Reasoning Engine endpoint."""
  # Set the headers and data
  headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json; charset=utf-8",
  }

  # Create the JSON payload from the query_str
  json_input = {"input": {"input": f"{query_str}"}}

  # Construct the URL
  _=load_dotenv(find_dotenv())

  PROJECT_ID = os.getenv('PROJECT_ID_INT') if os.getenv('PROJECT_ID_INT') else '982845833565'
  LOCATION_ID = os.getenv('AGENT_LOCATION') if os.getenv('AGENT_LOCATION') else 'us-central1'

  REASONING_ENGINE_ID = os.getenv('REASONING_ENGINE_ID') if os.getenv('REASONING_ENGINE_ID') else "6866054291275120640"
  url = f"https://{LOCATION_ID}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION_ID}/reasoningEngines/{REASONING_ENGINE_ID}:query"

  # Send the POST request
  r = requests.post(url, headers=headers, data=json.dumps(json_input))
  return r.json()

## TMF AIVA - Adhoc Testing

In [25]:
credentials, _ = google.auth.default()
request = google.auth.transport.requests.Request()
credentials.refresh(request)
token = credentials.token

In [26]:
queries = [
    "Describe ODF",
    "Can you give me the code for Product catalogue management version 4? use swagger gen tool",
    "Are TM Forum and MEF APIs the same?"
]

In [27]:
import json

experiment = "June05Adhoc"
batch = "v1"
bucket_name = "tmf_input_files"
source_file_name = "bulk_question_answering_reasoning_engine_summary.csv"
destination_file_name = f"bulk_question_answering_reasoning_engine_summary_result{experiment}_{batch}.csv"

def adhoc_testing(query, token, search_type="search"): # search_type = search | code_gen
  res = call_tmf_aiva_agent(queries, token)
  formatted_agent_response = parse_agent_response(res)
  with open(f'reasoning_agent_{search_type}_response.json', 'w') as fp:
    json.dump(res, fp)

  with open(f'reasoning_agent_{search_type}_formatted_response.json', 'w') as fp:
      json.dump(formatted_agent_response, fp)
  agent_response_dict = build_agent_response_dict(formatted_agent_response)
  df_agent_response_dict = pd.DataFrame([agent_response_dict])
  df_agent_response_dict.to_csv(f"gs://{bucket_name}/{destination_file_name}", sep=",", mode='a', index=False, header=True)


In [28]:
adhoc_testing(queries[0], token, "search")

In [ ]:
adhoc_testing("What is the role of CSPs in the coming years?", token, "search")

## TMF AIVA - Bulk QA testing

In [ ]:
import time
import pandas as pd
import random


experiment = "June05Bulk"
batch = "v1"
bucket_name = "tmf_input_files"
source_file_name = "bulk_question_answering_reasoning_engine_summary.csv"
destination_file_name = f"bulk_question_answering_reasoning_engine_summary_result{experiment}_{batch}.csv"

In [ ]:
def bulk_qa():
  df = pd.read_csv("gs://tmf_input_files/bulk_qa_mixed_questions.csv", header=0, dtype=str)
  rate_limiter = RateLimiter(max_calls=4, period=60)
  summary_df = pd.DataFrame()
  queries = []
  for index, row in df.iterrows():
    queries.append(row["Query"])

  random.shuffle(queries)
  try:
    for query in queries:
      with rate_limiter:
        start = time.time()
        agent_response = call_tmf_aiva_agent(query, token)
        agent_summary = agent_response["output"]
        response = False
        if agent_summary != "":
          response = True
        formatted_agent_response = parse_agent_response(agent_response)
        agent_response_dict = build_agent_response_dict(formatted_agent_response)
        df_agent_response_dict = pd.DataFrame([agent_response_dict])
        summary_df = pd.concat([summary_df, df_agent_response_dict], ignore_index=True)
        end = time.time()
        print(f"Query: {query}, output status: {response} - latency: {end - start}")
    summary_df.to_csv(f"gs://{bucket_name}/{destination_file_name}", sep=",", mode='a', index=False, header=True)
  except Exception as exc:
    summary_df.to_csv(f"gs://{bucket_name}/{destination_file_name}", sep=",", mode='a', index=False, header=True)
    print(exc)

In [ ]:
bulk_qa()

Query: how do I start building an autonomous network? - latency: 23.26177406311035
Query: Explain in more details how I would implement autonomous operations? - latency: 27.46544098854065
Query: How can I use ODA patterns to abstract Network components? - latency: 22.072493314743042
Query: How can I apply NaaS? - latency: 19.00199007987976
Query: Generate code in java for Trouble ticket API - latency: 6.074657201766968
Query: Generate code in java for product catalogue - latency: 6.12749457359314
Query: How do I start building an autonomous network? - latency: 29.284558057785034
Query: Is ODA architecture AI ready? - latency: 14.746080875396729
Query: When is DTW Mumbai? - latency: 10.882174968719482
Query: How do I apply AI to improve my autonomous operations? - latency: 19.565954208374023
Query: c++ code generation  for Product Ordering API - latency: 4.985942363739014
Query: I am trying to implement an ODA enabled architecture. How do I create a customer and set up their account? - 

In [ ]:
# errors faced

# {'error': {'code': 400, 'message': 'Reasoning Engine Execution failed. Error Details: {"detail":"Traceback (most recent call last):\\n  File \\"/usr/local/lib/python3.10/site-packages/google/api_core/grpc_helpers.py\\", line 170, in error_remapped_callable\\n    return _StreamingResponseIterator(\\n  File \\"/usr/local/lib/python3.10/site-packages/google/api_core/grpc_helpers.py\\", line 92, in __init__\\n    self._stored_first_result = next(self._wrapped)\\n  File \\"/usr/local/lib/python3.10/site-packages/grpc/_channel.py\\", line 543, in __next__\\n    return self._next()\\n  File \\"/usr/local/lib/python3.10/site-packages/grpc/_channel.py\\", line 969, in _next\\n    raise self\\ngrpc._channel._MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:\\n\\tstatus = StatusCode.INTERNAL\\n\\tdetails = \\"Internal error occurred.\\"\\n\\tdebug_error_string = \\"UNKNOWN:Error received from peer ipv4:173.194.196.95:443 {created_time:\\"2024-06-04T12:47:56.563121229+00:00\\", grpc_status:13, grpc_message:\\"Internal error occurred.\\"}\\"\\n>\\n\\nThe above exception was the direct cause of the following exception:\\n\\nTraceback (most recent call last):\\n  File \\"/code/app/api/factory/python_file_api_builder.py\\", line 109, in handler\\n    output = invocation_callable(**invocation_payload)\\n  File \\"/usr/local/lib/python3.10/site-packages/vertexai/preview/reasoning_engines/templates/langchain.py\\", line 399, in query\\n    self._runnable.invoke(input=input, config=config, **kwargs)\\n  File \\"/usr/local/lib/python3.10/site-packages/langchain/chains/base.py\\", line 163, in invoke\\n    raise e\\n  File \\"/usr/local/lib/python3.10/site-packages/langchain/chains/base.py\\", line 153, in invoke\\n    self._call(inputs, run_manager=run_manager)\\n  File \\"/usr/local/lib/python3.10/site-packages/langchain/agents/agent.py\\", line 1432, in _call\\n    next_step_output = self._take_next_step(\\n  File \\"/usr/local/lib/python3.10/site-packages/langchain/agents/agent.py\\", line 1138, in _take_next_step\\n    [\\n  File \\"/usr/local/lib/python3.10/site-packages/langchain/agents/agent.py\\", line 1138, in <listcomp>\\n    [\\n  File \\"/usr/local/lib/python3.10/site-packages/langchain/agents/agent.py\\", line 1166, in _iter_next_step\\n    output = self.agent.plan(\\n  File \\"/usr/local/lib/python3.10/site-packages/langchain/agents/agent.py\\", line 514, in plan\\n    for chunk in self.runnable.stream(inputs, config={\\"callbacks\\": callbacks}):\\n  File \\"/usr/local/lib/python3.10/site-packages/langchain_core/runnables/base.py\\", line 2875, in stream\\n    yield from self.transform(iter([input]), config, **kwargs)\\n  File \\"/usr/local/lib/python3.10/site-packages/langchain_core/runnables/base.py\\", line 2862, in transform\\n    yield from self._transform_stream_with_config(\\n  File \\"/usr/local/lib/python3.10/site-packages/langchain_core/runnables/base.py\\", line 1881, in _transform_stream_with_config\\n    chunk: Output = context.run(next, iterator)  # type: ignore\\n  File \\"/usr/local/lib/python3.10/site-packages/langchain_core/runnables/base.py\\", line 2826, in _transform\\n    for output in final_pipeline:\\n  File \\"/usr/local/lib/python3.10/site-packages/langchain_core/runnables/base.py\\", line 1282, in transform\\n    for ichunk in input:\\n  File \\"/usr/local/lib/python3.10/site-packages/langchain_core/runnables/base.py\\", line 4736, in transform\\n    yield from self.bound.transform(\\n  File \\"/usr/local/lib/python3.10/site-packages/langchain_core/runnables/base.py\\", line 1300, in transform\\n    yield from self.stream(final, config, **kwargs)\\n  File \\"/usr/local/lib/python3.10/site-packages/langchain_core/language_models/chat_models.py\\", line 249, in stream\\n    raise e\\n  File \\"/usr/local/lib/python3.10/site-packages/langchain_core/language_models/chat_models.py\\", line 229, in stream\\n    for chunk in self._stream(messages, stop=stop, **kwargs):\\n  File \\"/usr/local/lib/python3.10/site-packages/langchain_google_vertexai/chat_models.py\\", line 758, in _stream\\n    for response_chunk in response_iter:\\n  File \\"/usr/local/lib/python3.10/site-packages/vertexai/generative_models/_generative_models.py\\", line 572, in _generate_content_streaming\\n    response_stream = self._prediction_client.stream_generate_content(\\n  File \\"/usr/local/lib/python3.10/site-packages/google/cloud/aiplatform_v1beta1/services/prediction_service/client.py\\", line 2228, in stream_generate_content\\n    response = rpc(\\n  File \\"/usr/local/lib/python3.10/site-packages/google/api_core/gapic_v1/method.py\\", line 131, in __call__\\n    return wrapped_func(*args, **kwargs)\\n  File \\"/usr/local/lib/python3.10/site-packages/google/api_core/grpc_helpers.py\\", line 174, in error_remapped_callable\\n    raise exceptions.from_grpc_error(exc) from exc\\ngoogle.api_core.exceptions.InternalServerError: 500 Internal error occurred.\\n"}', 'status': 'FAILED_PRECONDITION'}}
